# Build data with proposition

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService

DATA_REPO = "BroDeadlines/TEST.edu_tdt_data"
SPLIT = "INDEX.medium_index_TDT"
provider = ProviderService()

In [ ]:
test_dataset = load_dataset(DATA_REPO, split= SPLIT)
test_dataset

In [ ]:
rows = test_dataset.filter(lambda e: e['url'] == "https://tuvanhocduong.tdtu.edu.vn/News")
rows

In [6]:
lengths = []
def count_len(row):
    lengths.append(len(row['content']))
    return

contents = []
def get_content(row):
    contents.append(row['content'])
    return

test_dataset.map(get_content)
len(lengths)
len(contents)

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

344

In [12]:
import numpy as np

data = np.array(lengths)

mean_value = np.mean(data)
print("The mean of the data is:", mean_value)

meandia_value = np.median(lengths)
print("The median of the data is:", meandia_value)

The mean of the data is: 670.2569444444445
The mean of the data is: 374.5


In [4]:
from src.utils.text_utils import create_splitter, try_split_texts
from src.chain.proposition import create_proposition_chain

splitter = create_splitter(chunk_size=400, overlap=50)
proponizer = create_proposition_chain(provider=provider)
# try_split_texts(texts=contents, splitter=splitter)


In [14]:
def build_data_func(row):
    content = row['content']
    splits = splitter.split_text(content)
    row['split'] = splits
    row['shards'] = len(splits)
    propositions = []
    for split in splits:
        try:
            pr_position = proponizer.invoke({"para": split})
            propositions.append(pr_position)
        except Exception:
            print(Exception)
    row['propositions'] = propositions
    return row

newds = test_dataset.map(build_data_func)

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

In [15]:
DATA_REPO = "BroDeadlines/TEST.proposition.edu_tdt_data"

newds.push_to_hub(DATA_REPO, split="medium_edu_tdt")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.proposition.edu_tdt_data/commit/728f5f60a7f8e6c61aa223b2c04af5c9a1c356d4', commit_message='Upload dataset', commit_description='', oid='728f5f60a7f8e6c61aa223b2c04af5c9a1c356d4', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
newds[0]['proppo']

{'content': 'Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.',
 'url': 'https://tuvanhocduong.tdtu.edu.vn/News',
 'doc_id': '573ca174-1b6d-11ef-a755-d38426455a06',
 'shards': 1,
 'splits': ['Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh 

## Build JSON list

In [1]:
def parse_error_json_list(inputdata):
    data = []
    rows = inputdata.split("\n")
    for r in rows:
        if len(r) < 10:
            print(r)
            continue
        tmp = r.strip()
        if tmp[-2:] == '",':
            tmp = tmp[:-2]
        if tmp[0] == '"':
            tmp = tmp[1:]
        if tmp[-1] == '"':
            tmp = tmp[:-1]
        data.append(tmp)
    return data

def parse_json(row):
    data = []
    data_list = row['propositions']
    for d in data_list:
        try:
            tmp_list = json.loads(d)
        except:
            # print("Can't parse")
            tmp_list = parse_error_json_list(d)
            # errors.extend(d)
            # continue
        data.extend(tmp_list)
    row['proposition_list'] = data
    return row

## Update DS

In [2]:
DATA_REPO = "BroDeadlines/TEST.edu_tdt_proposition_data"

test_dataset = load_dataset(DATA_REPO)

test_dataset

DatasetDict({
    propositon_medium_edu_tdt: Dataset({
        features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
        num_rows: 344
    })
    INDEX.medium_index_TDT: Dataset({
        features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
        num_rows: 344
    })
    INDEX.medium_index_TDT_clean: Dataset({
        features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
        num_rows: 344
    })
})

In [5]:
a = test_dataset['INDEX.medium_index_TDT_clean'][0]['propositions']
a

['[\n  "Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến.",\n  "Sinh viên nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến.",\n  "Sinh viên chọn học kỳ muốn gia hạn học phí.",\n  "Sinh viên chọn loại đơn cần nộp.",\n  "Sinh viên tiến hành điền các thông tin cần thiết.",\n  "Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không.",\n  "Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.",\n  "Nếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí.",\n  "Sinh viên đính kèm minh chứng khi nộp đơn gia hạn thời gian đóng học phí.",\n  "Sinh viên nộp đơn gia hạn thời gian đóng học phí về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí."\n]']

In [7]:
a = test_dataset['INDEX.medium_index_TDT'][0]['proposition_list']
b = "\n".join(a)
b

'Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến.\nSinh viên nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến.\nSinh viên chọn học kỳ muốn gia hạn học phí.\nSinh viên chọn loại đơn cần nộp.\nSinh viên tiến hành điền các thông tin cần thiết.\nSinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không.\nViệc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí.\nSinh viên đính kèm minh chứng khi nộp đơn gia hạn thời gian đóng học phí.\nSinh viên nộp đơn gia hạn thời gian đóng học phí về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.'

In [16]:
def clean_propositions(row):
    data = {**row}
    data['propositions'] = ["\n".join(row["proposition_list"])]
    return data
    
ds_new = test_dataset['INDEX.medium_index_TDT'].map(clean_propositions)

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

In [17]:
a = ds_new[0]['propositions']
a

['Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến.\nSinh viên nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến.\nSinh viên chọn học kỳ muốn gia hạn học phí.\nSinh viên chọn loại đơn cần nộp.\nSinh viên tiến hành điền các thông tin cần thiết.\nSinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không.\nViệc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí.\nSinh viên đính kèm minh chứng khi nộp đơn gia hạn thời gian đóng học phí.\nSinh viên nộp đơn gia hạn thời gian đóng học phí về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.']

In [18]:
ds_new.push_to_hub(DATA_REPO, split='INDEX.medium_index_TDT_clean')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.edu_tdt_proposition_data/commit/34e00f2c2b082b27cb8268c35fd0f0812b4a6c0e', commit_message='Upload dataset', commit_description='', oid='34e00f2c2b082b27cb8268c35fd0f0812b4a6c0e', pr_url=None, pr_revision=None, pr_num=None)

## Create split on Test.basic

In [43]:
EVAL = 'BroDeadlines/EVAL.IR_evaluation'

SPLIT = 'INDEX.medium_index_TDT'
test_dataset = load_dataset(EVAL, split=SPLIT)

test_dataset

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'evaluation', 'metadata'],
    num_rows: 144
})

In [7]:
ids = {}
def build_ids(row):
    ids[row['doc_id']] = 1
    return row

test_dataset.map(build_ids)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'evaluation', 'metadata'],
    num_rows: 20
})

In [8]:
REPO = 'BroDeadlines/TEST.edu_tdt_proposition_data'

SPLIT = 'INDEX.medium_index_TDT_clean'
repo_dataset = load_dataset(REPO, split=SPLIT)

repo_dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [33]:
repo_dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [36]:
repo_fil = repo_dataset.filter(lambda row: row['url'] == 'https://tuvanhocduong.tdtu.edu.vn/News').shuffle().select(range(10))
repo_fil

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 10
})

In [38]:
repo_qa = repo_dataset.filter(lambda row: row['url'] != 'https://tuvanhocduong.tdtu.edu.vn/News').shuffle().select(range(10))
repo_qa 

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 10
})

In [41]:
from datasets import concatenate_datasets

repo = concatenate_datasets([repo_qa, repo_fil])
repo

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 20
})

In [42]:
REPO = 'BroDeadlines/TEST.edu_tdt_proposition_data'

SPLIT = 'TEST.basic_index_TDT_clean'
repo.push_to_hub(REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.edu_tdt_proposition_data/commit/64d7e163f6d3ec0138a9e000baaf6452863cb881', commit_message='Upload dataset', commit_description='', oid='64d7e163f6d3ec0138a9e000baaf6452863cb881', pr_url=None, pr_revision=None, pr_num=None)

### Up to test data

In [44]:
ids = {}
def build_ids(row):
    ids[row['doc_id']] = 1
    return row

repo_fil.map(build_ids)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 10
})

In [47]:
a = test_dataset.filter(lambda e: e['doc_id'] in ids)

a

Filter:   0%|          | 0/144 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'evaluation', 'metadata'],
    num_rows: 10
})

In [48]:
EVAL = 'BroDeadlines/EVAL.IR_evaluation'

SPLIT = 'TEST.basic_raptor_TDT'
a.push_to_hub(EVAL, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/6.19k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/EVAL.IR_evaluation/commit/6d6ba39fa4ffa48b70b02d278342210f1a33b8fe', commit_message='Upload dataset', commit_description='', oid='6d6ba39fa4ffa48b70b02d278342210f1a33b8fe', pr_url=None, pr_revision=None, pr_num=None)